# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Procesamiento de Datos en Redes Sociales** </center>

---

**Proyecto Final**

**Fecha**: 13 mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez


**INTRODUCCIÓN Y DEFINICIÓN DEL PROBLEMA**

En la actualidad, las redes sociales se han consolidado como una fuente inagotable de datos generados en tiempo real por millones de usuarios en todo el mundo. Plataformas como Twitter e Instagram concentran interacciones constantes que reflejan opiniones, emociones, tendencias de consumo, comportamientos sociales y eventos relevantes en tiempo real. Aprovechar este flujo de información puede representar una ventaja estratégica para diversos sectores, como el marketing, la investigación social, la gestión de crisis, o incluso la predicción de fenómenos colectivos.

El objetivo de este proyecto es desarrollar una solución de análisis en tiempo real de datos provenientes de redes sociales, utilizando tecnologías de procesamiento distribuido como Apache Kafka y Apache Spark. La aplicación simula un entorno donde se generan y transmiten publicaciones de redes sociales de manera continua, con el fin de analizarlas en tiempo real para extraer métricas de interés, como usuarios más activos, mensajes más populares, tendencias por frecuencia de palabras y patrones de interacción.

El problema que se busca abordar es cómo gestionar, procesar y extraer valor de grandes volúmenes de datos no estructurados y en constante flujo. Las empresas y organizaciones frecuentemente enfrentan el reto de recibir información en tiempo real, pero carecen de mecanismos eficientes para procesarla sin demoras, asegurando al mismo tiempo la veracidad de los datos y obteniendo resultados útiles y accionables.

Para abordar este reto, se ha diseñado una arquitectura basada en el ecosistema Big Data que permite el procesamiento en tiempo real (streaming), el manejo de grandes volumenes de información (escalabilidad), la integración de fuentes de datos diversas (variedad). La solución se contruye a partir de productores que simulan publicaciones de Twitter e Instagram, enviadas mediante Kafka a distintos tópicos. Spark Structured Streaming actúa como el motor de procesamiento que consume, transforma y analiza estos flujos, permitiendo realizar cálculos y visualizaciones de manera inmediata.

Este proyecto busca demostrar cómo una arquitectura moderna de Big Data puede ser implementada para enfrentar los desafíos de las 5Vs del Big Data: Volumen, Velocidad, Veracidad y Valor. A través de este desarrollo, se pretende no solo evidenciar el funcionamiento técnico del sistema, sino también mostrar el valor práctica de contar con herramientas de análisis en tiempo real en un mundo cada vez más impulsado por los datos.

**ARQUITECTURA DEL SISTEMA**

System Architecture: Detailed description of the architecture used
(how all components are connected), including diagrams and expla-
nations of the real-time and batch processing components.

**5V JUSTIFICACIÓN**

5Vs Justification: Brief explanation of how the application meets
the 5Vs of Big Data:

– Volume: How the system handles large data volumes. Each
team needs to compute the size of each produced record to to
this analysis. Take as example the following Data Growth shown
in the following table:

– Velocity: The system’s ability to process streaming data in real-
time. The performance can be obtained by using the processedRowsPerSecond info obtained from the event progress
data (using QueryListeners).

– Variety: In this section, the report should include a description
of the schema that the application is handling. The schema
definition consists of the list of column names and data types
consumed by the PySpark application. The schema will depend
on the specific application chosen.

– Veracity: How the system ensures the quality and accuracy of
the data.

– Value: The practical insights or benefits of the columns stored
in the Database.

In [8]:
import findspark
findspark.init()

### Creacion de la conexión con el cluster de spark


Se crea una sesión de Spark con el paquete de Kafka incluido.

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Arriba-Linux-Proyecto-Final") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

### Crear el stream desde Kafka

Se hace la conexión a Kafka para escuchar los dos topicos. Kafka entrega los datos en formato binario, por lo que se debe convertirlos. Se crearon los tópicos directamente en la consola de kafka para ahí subscribirse a la lectura de datos.

In [10]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "78a305ddc318:9093") \
                .option("subscribePattern", ".*_topic") \
                .option("startingOffsets", "latest") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transformar los datos a columnas de strings

Se usa .withColumn para castear el string y luego descomponerlo para obtener colúmnas útiles. Ya que todo se está usando en strings, se tomó la decisión de usar cast("int") en el único dato diferente para simplificar el proceso.

In [11]:
from pyspark.sql.functions import split, col

kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

social_df = kafka_df.select(
    split(col("value_str"), " \\| ").alias("fields")
).select(
    col("fields")[0].alias("timestamp"),
    col("fields")[1].alias("platform"),
    col("fields")[2].alias("user"),
    col("fields")[3].alias("text"),
    col("fields")[4].cast("int").alias("likes")
)


### Escribir el stream en formato Parquet (Data Lake)

Se guarda el resultado del stream como archivos Parquet en un directorio.

In [ ]:
query = social_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/datalake/social_logs/") \
    .option("checkpointLocation", "/home/jovyan/notebooks/datalake/_checkpoints/") \
    .start()

query.awaitTermination(200)

25/05/13 02:57:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 02:57:50 WARN StreamingQueryManager: Stopping existing streaming query [id=736cf695-3fca-4dd5-bfa1-271f4661eafd, runId=c7bc72b0-c477-49ae-a193-05f8ae8cda5a], as a new run is being started.
25/05/13 02:57:58 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 02:57:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 7517 milliseconds


In [ ]:
df = spark.read.parquet("/home/jovyan/notebooks/datalake/social_logs/")
df.show(10, truncate=False)


+-------------------+---------+-------+-------------------------+-----+
|timestamp          |platform |user   |text                     |likes|
+-------------------+---------+-------+-------------------------+-----+
|2025-05-13 02:23:21|instagram|user_8 |My heart is full         |1939 |
|2025-05-13 02:23:21|instagram|user_25|Look at this outfit!     |2268 |
|2025-05-13 02:23:21|instagram|user_37|Throwback to last summer |2110 |
|2025-05-13 02:23:21|instagram|user_43|So in love with this meal|2920 |
|2025-05-13 02:23:21|instagram|user_10|Morning vibes            |80   |
|2025-05-13 02:23:21|instagram|user_39|Besties forever          |346  |
|2025-05-13 02:23:21|instagram|user_50|This place is magical    |1301 |
|2025-05-13 02:23:21|instagram|user_21|Beach days are the best  |1600 |
|2025-05-13 02:23:21|instagram|user_13|My heart is full         |755  |
|2025-05-13 02:23:21|instagram|user_24|Throwback to last summer |4326 |
+-------------------+---------+-------+-------------------------

### Preparación de Datos

In [ ]:
sc.stop()

NameError: name 'sc' is not defined